In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [ ]:
#实数序列数据预测问题，可以使用平方根误差
# def rmse(y_true,y_pred):
#     rmse=round(np.sqrt(np.average((y_pred-np.array(y_true))**2)),5)
#     return rmse


#这个评估标准是大赛规定的
def rmsle(y_true, y_pred):
    return 'RMSLE', round(np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))),5), False

# from sklearn.metrics import mean_squared_error
#也可以使用mse评价标准

In [ ]:
# ======按照出发城市进行分集存储后提取训练==============

#按照出发城市进行分集存储
def save_data(path,data):
    for i in range(0,7):
        ndata=data[data['o_city_code']==i]
        ndata.to_csv(path+'data_'+str(i)+'.csv',index=False)
        
path='../data/'
save_data(path,data)

gc.collect()

# =====将train=train_set+off_set,再train_set=tr_set+val_set===

In [ ]:
# class lightgbm.LGBMRegressor(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, 
#                              n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, 
#                              min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, 
#                              subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, 
#                              random_state=None, n_jobs=-1, silent=True, importance_type='split', **kwargs)

def get_xy(path,i):
    data=pd.read_csv(path+'data_'+str(i)+'.csv')
    train=data[data['cnt'].notnull()]
    test=data[data['cnt'].isnull()]
    print(train.shape,test.shape)
    
    train_xy,off_xy=train_test_split(train,test_size=0.1,random_state=1)
    tr,val=train_test_split(train_xy,test_size=0.15,random_state=1)
    
    drop_list=['cnt','o_city_code','o_city_code_hot']
    y_train=tr.cnt
    x_train=tr.drop(drop_list,axis=1)
    y_val=val.cnt
    x_val=val.drop(drop_list,axis=1)
    x_off=off_xy.drop(drop_list,axis=1)
    y_off=off_xy.cnt
    test_x=test.drop(drop_list,axis=1)
    return x_train,y_train,x_val,y_val,x_off,y_off,test_x

In [ ]:
def lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,i):
    gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=50,
                        learning_rate=0.05,
                        n_estimators=942)

    gbm.fit(x_train, y_train,
        eval_set=[(x_train,y_train),(x_val,y_val)],
        eval_metric='l1',
        early_stopping_rounds=5)

    y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
    y_eval=rmsle(y_val,y_pred)[1]
    # print('The rmse of prediction is:', mean_squared_error(y_val, y_pred) ** 0.5)
    print('The rmlse of prediction is:',y_eval)
    # print('Feature importances:', list(gbm.feature_importances_))

    y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
    y_off_eval=rmsle(y_off, y_pred_off)[1]
    # print('The rmse of prediction is:', mean_squared_error(y_off, y_pred_off) ** 0.5)
    print('The rmlse of prediction is:',y_off_eval)

    print("过拟合(-) 或 欠拟合(+)：",y_eval-y_off_eval)
    gc.collect()
    
    test['cnt']=gbm.predict(test_x,num_iteration=gbm.best_iteration_)
    test.to_csv(path+'test_'+str(i)+'.csv',index=False)
    
    return y_eval,y_off_eval

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,0)
rmsle0=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,0)

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,1)
rmsle1=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,1)

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,2)
rmsle2=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,2)

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,3)
rmsle3=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,3)

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,4)
rmsle4=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,4)

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,5)
rmsle5=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,5)

In [ ]:
x_train,y_train,x_val,y_val,x_off,y_off,test_x=get_xy(path,6)
rmsle6=lgb_train(x_train,y_train,x_val,y_val,x_off,y_off,test_x,6)

In [ ]:
mean_rmsle_val=(rmsle0[0]+rmsle1[0]+rmsle2[0]+rmsle3[0]+rmsle4[0]+rmsle5[0]+rmsle6[0])/7
mean_rmsle_off=(rmsle0[1]+rmsle1[1]+rmsle2[1]+rmsle3[1]+rmsle4[1]+rmsle5[1]+rmsle6[1])/7

# =======进行StratifiedKFold=10折===============

In [ ]:
from sklearn.model_selection import StratifiedKFold
k=5
skf=StratifiedKFold(n_splits=k,shuffle=True,random_state=1)

In [ ]:
def get_xy(path,i):
    data=pd.read_csv(path+'data_'+str(i)+'.csv')
    train=data[data['cnt'].notnull()]
    test=data[data['cnt'].isnull()]
    print(train.shape,test.shape)
    
    train_xy,off_xy=train_test_split(train,test_size=0.1,random_state=1)
    drop_list=['cnt','o_city_code','o_city_code_hot']   
    y=train_xy.cnt
    X=train_xy.drop(drop_list,axis=1)
    
    y_off=off_xy.cnt
    x_off=off_xy.drop(drop_list,axis=1)
    
    test_x=test.drop(drop_list,axis=1)
    
    return X,y,x_off,y_off,test_x

def lgb_train_k(X,y,x_off,y_off,test_x,k,i):
    for i,(train_index,test_index) in enumerate(skf.split(X,y)):
        print("Fold=%s"%(i))
        gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=50,
                        learning_rate=0.05,
                        n_estimators=942)
        gbm.fit(X[train_index],y[train_index],
                eval_set=[(X[train_index],y[train_index]),(X[test_index],y[test_index])],
               eval_metric='l1',
               early_stopping_rounds=10)
        
        y_pred=gbm.predict(X[test_index],num_iteration=gbm.best_iteration_)
        rmsle=rmsle(y[test_index],y_pred)[1]
        
        y_off_pred=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
        rmsle_off=rmsle(y_off,y_off_pred)[1]
        
        test['cnt']=gbm.predict(test_x,num_iteration=gbm.best_iteration_)
        test.to_csv(path+'testk_'+str(i)+'.csv',index=False)
        
        return rmsle,rmsle_off

In [ ]:
# print(test0[test0['cnt']>=0].o_city_code.value_counts())

# print(test0[test0['cnt']<0].o_city_code.value_counts())

# tmp=train[train['o_city_code']==0][train['month']==3][train['day']<=15].groupby('day').cnt.count().reset_index()
# print(tmp)
# print(tmp.cnt.sum())

#对于训练出来的负数，再后面整合的时候去掉

In [ ]:
#暴力搜索最优参数,感觉不太理想，这一版本暂且不用
# estimator = lgb.LGBMRegressor(num_leaves=31)
# param_grid = {
#     'learning_rate':[0.01,0.1,1],
#     'n_estimators': [60,65,70] }
 
# gbm = GridSearchCV(estimator, param_grid)
# gbm.fit(x_train,y_train)

# print('Best parameters found by grid search are:', gbm.best_params_)
# print('Best parameters found by grid search are:', gbm.best_score_)
# gc.collect()

#1,80,0.9644001175652634
#1,60,0.9612318411124592


#最好的参数的gbm
# gbm_best = lgb.LGBMRegressor(objective='regression',
#                         num_leaves=31,
#                         learning_rate=1,
#                         n_estimators=60)

# gbm_best.fit(x_train, y_train,
#         eval_set=[(x_train,y_train),(x_val,y_val)],
#         eval_metric='l1',
#         early_stopping_rounds=5)

# y_pred = gbm_best.predict(x_val, num_iteration=gbm_best.best_iteration_)
# print('The rmse of prediction is:', mean_squared_error(y_val, y_pred) ** 0.5)
# print('The rmlse of prediction is:',rmsle(y_val,y_pred)[1])

# y_offline=gbm_best.predict(offline_x,num_iteration=gbm_best.best_iteration_)
# print('The rmse of prediction is:', mean_squared_error(offline_y,y_offline) ** 0.5)
# print('The rmlse of prediction is:',rmsle(offline_y,y_offline)[1])

In [ ]:
### ============================<font color=red>try on all data<font>========================
#没有单独用城市来训练的效果好，故舍弃

# feature=['year','month','day','o_city_code','o_district_code','d_city_code','d_district_code']

# train_xy,off_xy=train_test_split(train,test_size=0.1,random_state=1)
# tr_xy,val_xy=train_test_split(train_xy,test_size=0.2,random_state=1)
# x_train=tr_xy[feature]
# y_train=tr_xy.cnt
# x_val=val_xy[feature]
# y_val=val_xy.cnt
# x_off=off_xy[feature]
# y_off=off_xy.cnt

# test_x=test[feature]

# gbm = lgb.LGBMRegressor(objective='regression',
#                         num_leaves=50,
#                         learning_rate=0.01,
#                         n_estimators=3000)

# gbm.fit(x_train, y_train,
#         eval_set=[(x_train,y_train),(x_val,y_val)],
#         eval_metric='l1',
#         early_stopping_rounds=100)

# y_pred = gbm.predict(x_val, num_iteration=gbm.best_iteration_)
# y_eval=rmsle(y_val,y_pred)[1]
# # print('The rmse of prediction is:', mean_squared_error(y_val, y_pred) ** 0.5)
# print('The rmlse of prediction is:',y_eval)
# # print('Feature importances:', list(gbm.feature_importances_))

# y_pred_off=gbm.predict(x_off,num_iteration=gbm.best_iteration_)
# y_off_eval=rmsle(y_off, y_pred_off)[1]
# # print('The rmse of prediction is:', mean_squared_error(y_off, y_pred_off) ** 0.5)
# print('The rmlse of prediction is:',y_off_eval)

# print("过拟合(-) 或 欠拟合(+)：",y_eval-y_off_eval)
# gc.collect()